In [5]:
import pandas as pd
import pickle
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /home/msimon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [351]:
#open data
with open('data.pkl', 'rb') as f:
    df = pickle.load(f)

In [352]:
#get the character length of texts
df['text_len'] = df['text'].astype(str).apply(len)
df.sort_values(by = 'text_len', ascending = False)

In [354]:
#clean the data from scratch
#remove url's from text
#remove phone numbers from text
def clean_url(text):
    return re.sub(r'http\S+', ' ', text)
def clean_email(text):
    return re.sub(r'[\w\.-]+@[\w\.-]+', ' ', text)
def clean_phone_n(text):
    return re.sub(r'^((\+|00(\s|\s?\-\s?)?)31(\s|\s?\-\s?)?(\(0\)[\-\s]?)?|0)[1-9]((\s|\s?\-\s?)?[0-9])((\s|\s?-\s?)?[0-9])((\s|\s?-\s?)?[0-9])\s?[0-9]\s?[0-9]\s?[0-9]\s?[0-9]\s?[0-9]$', ' ', text)
df['text_1'] = df['text'].apply(clean_url)
df['text_1'] = df['text_1'].apply(clean_email)
df['text_1'] = df['text_1'].apply(clean_phone_n)

In [356]:
#check len text once again we see that max is 4000 characters now
df['text_len'] = df['text_1'].astype(str).apply(len)
df.sort_values(by = 'text_len', ascending = False)

,date,url,source,type,ids,message_id,links,text,nolink_text,noemoji_text,spaced_text,text_len,text_1
30642,2021-03-29 19:52:52+00:00,https://t.me/vriendenvangeertwilders,vriendenvangeertwilders,chat,9.553423e+08,1246.0,https://en.m.wikipedia.org/w/index.php?title=E...,References\n[Edit](https://en.m.wikipedia.org/...,references\n[edit](,references\n[edit](,references\n[edit](,4431,"References\n[Edit]( \n** 1 ^** Horton, Richard..."
1651023,2021-03-29 14:54:12+00:00,https://t.me/nederlandsverzet21,nederlandsverzet21,channel,1.231903e+09,27057.0,https://t.me/QPatriots/35813)1B,🦁 Table of Contents▫️Group Rules▫️Resources ...,🦁 table of contents▫️group rules▫️resources ...,:lion: table of contents:white_small_square:gr...,:lion : table of contents :white_small_square ...,4286,🦁 Table of Contents▫️Group Rules▫️Resources ...
844227,2020-06-03 08:21:16+00:00,https://t.me/Kletschat_burgers_tegen_onrecht,Kletschat_burgers_tegen_onrecht,chat,8.254746e+08,38952.0,NaN,**IkGebruikMijnStem\n**`Ik stem niet in met ee...,**ikgebruikmijnstem\n**`ik stem niet in met ee...,**ikgebruikmijnstem\n**`ik stem niet in met ee...,**ikgebruikmijnstem\n**`ik stem niet in met ee...,4136,**IkGebruikMijnStem\n**`Ik stem niet in met ee...
48977,2020-03-05 23:54:34+00:00,https://t.me/FvDgroup,FvDgroup,chat,1.472674e+07,452.0,https://fvd.us3.list-manage.com/track/click?u=...,**Beste FVD-er!**\n**Op 20 maart is Forum voor...,**beste fvd-er!**\n**op 20 maart is forum voor...,**beste fvd-er!**\n**op 20 maart is forum voor...,**beste fvd-er!**\n**op 20 maart is forum voor...,4132,**Beste FVD-er!**\n**Op 20 maart is Forum voor...
1405264,2020-06-03 22:58:42+00:00,https://t.me/wakkerenchat,wakkerenchat,chat,3.200827e+08,44442.0,NaN,PUTTING THE EARTH INSIDE A HIGH-SPEED COMPUTER...,putting the earth inside a high-speed computer...,putting the earth inside a high-speed computer...,putting the earth inside a high-speed computer...,4130,PUTTING THE EARTH INSIDE A HIGH-SPEED COMPUTER...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
871756,2020-02-07 22:43:24+00:00,https://t.me/Kletschat_burgers_tegen_onrecht,Kletschat_burgers_tegen_onrecht,chat,1.000296e+09,9572.0,NaN,🤣,🤣,:rolling_on_the_floor_laughing:,:rolling_on_the_floor_laughing :,1,🤣
1376704,2020-07-28 19:30:48+00:00,https://t.me/wakkerenchat,wakkerenchat,chat,1.220709e+09,87354.0,NaN,😂,😂,:face_with_tears_of_joy:,:face_with_tears_of_joy :,1,😂
1376792,2020-07-28 18:41:45+00:00,https://t.me/wakkerenchat,wakkerenchat,chat,6.721698e+08,87253.0,NaN,👍,👍,:thumbs_up:,:thumbs_up :,1,👍
1376830,2020-07-28 18:19:24+00:00,https://t.me/wakkerenchat,wakkerenchat,chat,8.214925e+08,87208.0,NaN,😄,😄,:grinning_face_with_smiling_eyes:,:grinning_face_with_smiling_eyes :,1,😄


In [357]:
#check the number of rows that contain text longer than 384 word tokens  = ~ 2000 characters
df[df.text_len > 2000].shape[0]
#11,483 docs that are longer than about 384 words
#we will need to chop the first part off

11483

In [358]:
#25,783 rows with less than 2 tokens 
df[df.text_len <2].shape[0] #but this includes emojis which we would like to keep

25783

In [359]:
#we perform sentence tokenixation on texts longer than 2000 characters. - given that this is about the equivalent of 384 words/tokens bert can ahndle
df['text_2'] = df['text_1'].astype(str).apply(lambda x: sent_tokenize(x) if len(x) > 2000 else x) # using the NLTK SentenceTokenizer
df_tmp = pd.DataFrame(df['text_2'].explode())
df = df_tmp.join(df.drop('text_2', axis=1))

In [362]:
#only keep the first 2000 characters of texts
df['text_3'] = df['text_2'].astype(str).apply(lambda x: x[0:2000] if len(x) > 2000 else x) 

In [364]:
#remove single punctuation if len of text is lower than 2 tokens but keep emoticons
df['texts'] = df['text_3'].astype(str).apply(lambda x: re.sub(r'(?<=\w)[^\s\w](?![^\s\w])',  '', x) if len(x)< 2 else x) 

In [365]:
#check len of str again
#check len text once again
df['text_len'] = df['texts'].astype(str).apply(len)
df.sort_values(by = 'text_len', ascending = False) 

,text_2,date,url,source,type,ids,message_id,links,text,nolink_text,noemoji_text,spaced_text,text_len,text_1,text_3,texts
718501,"de rest van Nederland verpesten, kansloos en s...",2021-01-26 18:57:26+00:00,https://t.me/rellen023,rellen023,chat,1.615468e+09,1740.0,NaN,"de rest van Nederland verpesten, kansloos en s...","de rest van nederland verpesten, kansloos en s...","de rest van nederland verpesten, kansloos en s...","de rest van nederland verpesten, kansloos en s...",2000,"de rest van Nederland verpesten, kansloos en s...","de rest van Nederland verpesten, kansloos en s...","de rest van Nederland verpesten, kansloos en s..."
718488,"de rest van Nederland verpesten, kansloos en s...",2021-01-26 18:57:35+00:00,https://t.me/rellen023,rellen023,chat,1.604510e+09,1756.0,NaN,"de rest van Nederland verpesten, kansloos en s...","de rest van nederland verpesten, kansloos en s...","de rest van nederland verpesten, kansloos en s...","de rest van nederland verpesten, kansloos en s...",2000,"de rest van Nederland verpesten, kansloos en s...","de rest van Nederland verpesten, kansloos en s...","de rest van Nederland verpesten, kansloos en s..."
419983,I just received a list of politicians who’ve d...,2021-02-08 20:08:28+00:00,https://t.me/DeBataafseRepubliek,DeBataafseRepubliek,chat,1.525233e+09,46026.0,NaN,I just received a list of politicians who’ve d...,i just received a list of politicians who’ve d...,i just received a list of politicians who’ve d...,i just received a list of politicians who’ve d...,2000,I just received a list of politicians who’ve d...,I just received a list of politicians who’ve d...,I just received a list of politicians who’ve d...
718643,"de rest van Nederland verpesten, kansloos en s...",2021-01-26 18:54:06+00:00,https://t.me/rellen023,rellen023,chat,1.643539e+09,1573.0,NaN,"de rest van Nederland verpesten, kansloos en s...","de rest van nederland verpesten, kansloos en s...","de rest van nederland verpesten, kansloos en s...","de rest van nederland verpesten, kansloos en s...",2000,"de rest van Nederland verpesten, kansloos en s...","de rest van Nederland verpesten, kansloos en s...","de rest van Nederland verpesten, kansloos en s..."
718644,"de rest van Nederland verpesten, kansloos en s...",2021-01-26 18:53:59+00:00,https://t.me/rellen023,rellen023,chat,1.643539e+09,1572.0,NaN,"de rest van Nederland verpesten, kansloos en s...","de rest van nederland verpesten, kansloos en s...","de rest van nederland verpesten, kansloos en s...","de rest van nederland verpesten, kansloos en s...",2000,"de rest van Nederland verpesten, kansloos en s...","de rest van Nederland verpesten, kansloos en s...","de rest van Nederland verpesten, kansloos en s..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26824,🥊,2021-01-28 15:27:05+00:00,https://t.me/wilderspvv,wilderspvv,chat,1.515136e+09,1457.0,NaN,🥊,🥊,:boxing_glove:,:boxing_glove :,1,🥊,🥊,🥊
967948,😂,2021-05-26 09:40:25+00:00,https://t.me/klokkenchat,klokkenchat,chat,1.673539e+09,331498.0,NaN,😂,😂,:face_with_tears_of_joy:,:face_with_tears_of_joy :,1,😂,😂,😂
769784,😋,2020-11-06 01:03:41+00:00,https://t.me/samen1voornl,samen1voornl,chat,8.788742e+08,64149.0,NaN,😋,😋,:face_savoring_food:,:face_savoring_food :,1,😋,😋,😋
1311948,R,2021-01-20 17:27:19+00:00,https://t.me/wakkerenchat,wakkerenchat,chat,1.478607e+09,190594.0,NaN,R,r,r,r,1,R,R,R


In [145]:
# Show the distribution of headline length 
#sns.displot(df.text_len, kde=False)

In [367]:
df= df.sort_values(by = 'date', ascending = True)

In [409]:
df_final = df[['date', 'url', 'source', 'type', 'message_id', 'links', 'texts']].copy()

In [426]:
#save dataset only with columns that matter
with open('clean_df.pkl', 'wb') as f:
    pickle.dump(df_final, f)